In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install py7zr

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 88.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 13.9 MB/s eta 0:00:00


In [2]:
import py7zr

with py7zr.SevenZipFile('/content/drive/MyDrive/data.7z', mode='r') as archive:
    archive.extractall(path='/content/data')


In [3]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [4]:
# Image Preparation
from image_processor import ImageProcessor

csv_path = "New_labels.csv"
image_folder = "/content/data/data/"

processor = ImageProcessor(csv_path, image_folder)
img_data, labels = processor.process_images()




In [5]:
# Data Preparation
from data_preprocessor import DataPreprocessor
preprocessor = DataPreprocessor(img_data, labels)
train_images, val_images, train_labels, val_labels = preprocessor.preprocess_data()


In [6]:
# Model Training
from speaker_agent import SpeakerAgent
agent = SpeakerAgent(train_images, train_labels, val_images, val_labels)
model = agent.create_model()


Epoch 1/30
400/400 [==============================] - 31s 44ms/step - loss: 3.9071 - accuracy: 0.1043 - val_loss: 10.8552 - val_accuracy: 0.0066
Epoch 2/30
400/400 [==============================] - 18s 44ms/step - loss: 2.3627 - accuracy: 0.2868 - val_loss: 2.2927 - val_accuracy: 0.3141
Epoch 3/30
400/400 [==============================] - 17s 43ms/step - loss: 1.7930 - accuracy: 0.4106 - val_loss: 1.3190 - val_accuracy: 0.5634
Epoch 4/30
400/400 [==============================] - 17s 43ms/step - loss: 1.4781 - accuracy: 0.5002 - val_loss: 1.1275 - val_accuracy: 0.6497
Epoch 5/30
400/400 [==============================] - 17s 43ms/step - loss: 1.2582 - accuracy: 0.5714 - val_loss: 0.8571 - val_accuracy: 0.7425
Epoch 6/30
400/400 [==============================] - 17s 43ms/step - loss: 1.1332 - accuracy: 0.6076 - val_loss: 0.6520 - val_accuracy: 0.8003
Epoch 7/30
400/400 [==============================] - 17s 43ms/step - loss: 1.0190 - accuracy: 0.6547 - val_loss: 0.5556 - val_accuracy

In [7]:
# Define the path to save the model
from tensorflow.keras.models import load_model
model_path = 'speaker_agent.h5'

model.save(model_path)
loaded_model = load_model(model_path)

# Game Round 1

In [8]:
# Usage
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
from image_predictor import ImagePredictor
# Load the Excel file into a pandas DataFrame
data = pd.read_csv("New_labels.csv")

# Extract the filenames and labels into separate arrays
filenames = data['Filenames'].values
shapes = data['labels'].values
shapes = np.array(shapes)
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(shapes)
file_path = "test_image.xlsx"
test_img_dir = "/content/drive/MyDrive/IMAGE test/"

from listener_agent import ListenerAgent

test_data = pd.read_excel(file_path)
test_img = test_data['Filepath']
target_label = test_data['shapes']
data_path = "/content/data/data/"


test_data = pd.read_excel(file_path)
test_img = test_data['Filepath']
target_label = test_data['shapes']


# Create an instance of ImagePredictor with appropriate arguments
image_predictor = ImagePredictor(test_img_dir, loaded_model, label_encoder)

# Call the predict_labels method to get the predicted labels
predicted_labels = image_predictor.predict_labels(test_img, target_label)


1/1 [==============================] - 0s 19ms/step


In [9]:
# Usage example:
from Game_env_listerner_model2 import GameEnvironmentListerModel
listener_agent = ListenerAgent(data_path, filenames, shapes)
game_env_lister_model = GameEnvironmentListerModel(test_img_dir, test_img, predicted_labels, listener_agent, data_path, filenames, shapes)


In [10]:
lis_image, lis_pred = game_env_lister_model.get_listener_model_predictions()

# Round 1 reward Accuracy

In [11]:
from CollaborativeReward import CollaborativeReward
collab_reward_calculator = CollaborativeReward(predicted_labels,target_label, lis_pred)
collab_reward, true_conditions, false_conditions = collab_reward_calculator.calculate_reward()


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [32]:
print(collab_reward)
collab_reward_percent = collab_reward/len(test_img)
print(collab_reward_percent)

8
0.1


# Game round 2

In [13]:
from policy_updater import ModelUpdater
import tensorflow as tf

# Create an instance of the ModelUpdater class
model_updater = ModelUpdater(weights_path='speaker_agent.h5')

# Call the update_model method
model, lr = model_updater.update_model(loaded_model, collab_reward_percent, train_images, train_labels, val_images, val_labels, max_epochs=30)

# Save the updated model
model_path = 'speaker_agent_updated_1.h5'
model.save(model_path)

# Load the updated model
loaded_model = tf.keras.models.load_model(model_path)


# Create an instance of ImagePredictor with appropriate arguments
image_predictor = ImagePredictor(test_img_dir, loaded_model, label_encoder)

# Call the predict_labels method to get the predicted labels
predicted_labels = image_predictor.predict_labels(test_img, target_label)
listener_agent = ListenerAgent(data_path, filenames, shapes)
game_env_lister_model = GameEnvironmentListerModel(test_img_dir, test_img, predicted_labels, listener_agent, data_path, filenames, shapes)
lis_image, lis_pred = game_env_lister_model.get_listener_model_predictions()

collab_reward_calculator = CollaborativeReward(predicted_labels,target_label, lis_pred)
collab_reward, true_conditions, false_conditions = collab_reward_calculator.calculate_reward()



Epoch 1/30
400/400 [==============================] - 21s 45ms/step - loss: 0.2755 - accuracy: 0.9075 - val_loss: 0.0381 - val_accuracy: 0.9947
Epoch 2/30
400/400 [==============================] - 17s 43ms/step - loss: 0.2505 - accuracy: 0.9157 - val_loss: 0.0487 - val_accuracy: 0.9906
Epoch 3/30
400/400 [==============================] - 17s 43ms/step - loss: 0.2492 - accuracy: 0.9203 - val_loss: 0.0312 - val_accuracy: 0.9953
Epoch 4/30
400/400 [==============================] - 18s 44ms/step - loss: 0.2408 - accuracy: 0.9208 - val_loss: 0.0378 - val_accuracy: 0.9937
Epoch 5/30
400/400 [==============================] - 17s 43ms/step - loss: 0.2231 - accuracy: 0.9258 - val_loss: 0.0333 - val_accuracy: 0.9956
Epoch 6/30
400/400 [==============================] - 18s 44ms/step - loss: 0.2236 - accuracy: 0.9276 - val_loss: 0.0330 - val_accuracy: 0.9947
Epoch 7/30
400/400 [==============================] - 17s 44ms/step - loss: 0.2243 - accuracy: 0.9269 - val_loss: 0.0659 - val_accuracy:

In [34]:
print(collab_reward)
collab_reward_percent = collab_reward/len(test_img)
print(collab_reward_percent)

9
0.1125


# Game Round 3

In [15]:
# Create an instance of the ModelUpdater class
model_updater = ModelUpdater(weights_path='speaker_agent_updated_1.h5')

# Call the update_model method
model, lr = model_updater.update_model(loaded_model, collab_reward_percent, train_images, train_labels, val_images, val_labels, max_epochs=30)

# Save the updated model
model_path = 'speaker_agent_updated_2.h5'
model.save(model_path)

# Load the updated model
loaded_model = tf.keras.models.load_model(model_path)

# Create an instance of ImagePredictor with appropriate arguments
image_predictor = ImagePredictor(test_img_dir, loaded_model, label_encoder)

# Call the predict_labels method to get the predicted labels
predicted_labels = image_predictor.predict_labels(test_img, target_label)
listener_agent = ListenerAgent(data_path, filenames, shapes)
game_env_lister_model = GameEnvironmentListerModel(test_img_dir, test_img, predicted_labels, listener_agent, data_path, filenames, shapes)
lis_image, lis_pred = game_env_lister_model.get_listener_model_predictions()

collab_reward_calculator = CollaborativeReward(predicted_labels,target_label, lis_pred)
collab_reward, true_conditions, false_conditions = collab_reward_calculator.calculate_reward()


Epoch 1/30
400/400 [==============================] - 21s 43ms/step - loss: 0.1485 - accuracy: 0.9502 - val_loss: 0.0292 - val_accuracy: 0.9959
Epoch 2/30
400/400 [==============================] - 17s 42ms/step - loss: 0.1416 - accuracy: 0.9530 - val_loss: 0.0291 - val_accuracy: 0.9966
Epoch 3/30
400/400 [==============================] - 17s 43ms/step - loss: 0.1362 - accuracy: 0.9561 - val_loss: 0.0301 - val_accuracy: 0.9953
Epoch 4/30
400/400 [==============================] - 17s 43ms/step - loss: 0.1367 - accuracy: 0.9544 - val_loss: 0.0316 - val_accuracy: 0.9937
Epoch 5/30
400/400 [==============================] - 17s 42ms/step - loss: 0.1482 - accuracy: 0.9521 - val_loss: 0.0327 - val_accuracy: 0.9950
Epoch 6/30
400/400 [==============================] - 17s 42ms/step - loss: 0.1407 - accuracy: 0.9530 - val_loss: 0.0296 - val_accuracy: 0.9950
Epoch 7/30
400/400 [==============================] - 17s 43ms/step - loss: 0.1412 - accuracy: 0.9535 - val_loss: 0.5934 - val_accuracy:

In [35]:
print(collab_reward)
collab_reward_percent = collab_reward/len(test_img)
print(collab_reward_percent)

9
0.1125


# Game round 4

In [17]:
# Create an instance of the ModelUpdater class
model_updater = ModelUpdater(weights_path='speaker_agent_updated_2.h5')

# Call the update_model method
model, lr = model_updater.update_model(loaded_model, collab_reward_percent, train_images, train_labels, val_images, val_labels, max_epochs=30)

# Save the updated model
model_path = 'speaker_agent_updated_3.h5'
model.save(model_path)

# Load the updated model
loaded_model = tf.keras.models.load_model(model_path)

# Create an instance of ImagePredictor with appropriate arguments
image_predictor = ImagePredictor(test_img_dir, loaded_model, label_encoder)

# Call the predict_labels method to get the predicted labels
predicted_labels = image_predictor.predict_labels(test_img, target_label)
listener_agent = ListenerAgent(data_path, filenames, shapes)
game_env_lister_model = GameEnvironmentListerModel(test_img_dir, test_img, predicted_labels, listener_agent, data_path, filenames, shapes)
lis_image, lis_pred = game_env_lister_model.get_listener_model_predictions()

collab_reward_calculator = CollaborativeReward(predicted_labels,target_label, lis_pred)
collab_reward, true_conditions, false_conditions = collab_reward_calculator.calculate_reward()



Epoch 1/30
400/400 [==============================] - 21s 44ms/step - loss: 0.1190 - accuracy: 0.9602 - val_loss: 0.0321 - val_accuracy: 0.9950
Epoch 2/30
400/400 [==============================] - 17s 43ms/step - loss: 0.1103 - accuracy: 0.9630 - val_loss: 0.0738 - val_accuracy: 0.9819
Epoch 3/30
400/400 [==============================] - 17s 42ms/step - loss: 0.1124 - accuracy: 0.9634 - val_loss: 0.0267 - val_accuracy: 0.9959
Epoch 4/30
400/400 [==============================] - 18s 44ms/step - loss: 0.1085 - accuracy: 0.9645 - val_loss: 0.0287 - val_accuracy: 0.9950
Epoch 5/30
400/400 [==============================] - 17s 43ms/step - loss: 0.1090 - accuracy: 0.9643 - val_loss: 0.0304 - val_accuracy: 0.9944
Epoch 6/30
400/400 [==============================] - 17s 42ms/step - loss: 0.1059 - accuracy: 0.9655 - val_loss: 0.0292 - val_accuracy: 0.9953
Epoch 7/30
400/400 [==============================] - 17s 42ms/step - loss: 0.1151 - accuracy: 0.9627 - val_loss: 0.0354 - val_accuracy:

In [37]:
print(collab_reward)
collab_reward_percent = collab_reward/len(test_img)
print(collab_reward_percent)

10
0.125


# Game round 5

In [19]:
# Create an instance of the ModelUpdater class
model_updater = ModelUpdater(weights_path='speaker_agent_updated_3.h5')

# Call the update_model method
model, lr = model_updater.update_model(loaded_model, collab_reward_percent, train_images, train_labels, val_images, val_labels, max_epochs=30)

# Save the updated model
model_path = 'speaker_agent_updated_4.h5'
model.save(model_path)

# Load the updated model
loaded_model = tf.keras.models.load_model(model_path)

# Create an instance of ImagePredictor with appropriate arguments
image_predictor = ImagePredictor(test_img_dir, loaded_model, label_encoder)

# Call the predict_labels method to get the predicted labels
predicted_labels = image_predictor.predict_labels(test_img, target_label)
listener_agent = ListenerAgent(data_path, filenames, shapes)
game_env_lister_model = GameEnvironmentListerModel(test_img_dir, test_img, predicted_labels, listener_agent, data_path, filenames, shapes)
lis_image, lis_pred = game_env_lister_model.get_listener_model_predictions()

collab_reward_calculator = CollaborativeReward(predicted_labels,target_label, lis_pred)
collab_reward, true_conditions, false_conditions = collab_reward_calculator.calculate_reward()



Epoch 1/30
400/400 [==============================] - 21s 43ms/step - loss: 0.0967 - accuracy: 0.9674 - val_loss: 0.0293 - val_accuracy: 0.9959
Epoch 2/30
400/400 [==============================] - 17s 43ms/step - loss: 0.1026 - accuracy: 0.9653 - val_loss: 0.0362 - val_accuracy: 0.9944
Epoch 3/30
400/400 [==============================] - 17s 42ms/step - loss: 0.0900 - accuracy: 0.9688 - val_loss: 0.0305 - val_accuracy: 0.9953
Epoch 4/30
400/400 [==============================] - 17s 43ms/step - loss: 0.0961 - accuracy: 0.9696 - val_loss: 0.0316 - val_accuracy: 0.9953
Epoch 5/30
400/400 [==============================] - 17s 42ms/step - loss: 0.0955 - accuracy: 0.9695 - val_loss: 0.0305 - val_accuracy: 0.9953
Epoch 6/30
400/400 [==============================] - 17s 42ms/step - loss: 0.0967 - accuracy: 0.9688 - val_loss: 0.0338 - val_accuracy: 0.9947
Epoch 7/30
400/400 [==============================] - 17s 42ms/step - loss: 0.0910 - accuracy: 0.9702 - val_loss: 0.1137 - val_accuracy:

In [39]:
print(collab_reward)
collab_reward_percent = collab_reward/len(test_img)
print(collab_reward_percent)

11
0.1375


# Game Round 6

In [21]:
# Create an instance of the ModelUpdater class
model_updater = ModelUpdater(weights_path='speaker_agent_updated_4.h5')

# Call the update_model method
model, lr = model_updater.update_model(loaded_model, collab_reward_percent, train_images, train_labels, val_images, val_labels, max_epochs=30)

# Save the updated model
model_path = 'speaker_agent_updated_5.h5'
model.save(model_path)

# Load the updated model
loaded_model = tf.keras.models.load_model(model_path)

# Create an instance of ImagePredictor with appropriate arguments
image_predictor = ImagePredictor(test_img_dir, loaded_model, label_encoder)

# Call the predict_labels method to get the predicted labels
predicted_labels = image_predictor.predict_labels(test_img, target_label)
listener_agent = ListenerAgent(data_path, filenames, shapes)
game_env_lister_model = GameEnvironmentListerModel(test_img_dir, test_img, predicted_labels, listener_agent, data_path, filenames, shapes)
lis_image, lis_pred = game_env_lister_model.get_listener_model_predictions()

collab_reward_calculator = CollaborativeReward(predicted_labels,target_label, lis_pred)
collab_reward, true_conditions, false_conditions = collab_reward_calculator.calculate_reward()



Epoch 1/30
400/400 [==============================] - 21s 43ms/step - loss: 0.0901 - accuracy: 0.9688 - val_loss: 0.0348 - val_accuracy: 0.9953
Epoch 2/30
400/400 [==============================] - 17s 43ms/step - loss: 0.0825 - accuracy: 0.9723 - val_loss: 0.0419 - val_accuracy: 0.9925
Epoch 3/30
400/400 [==============================] - 17s 42ms/step - loss: 0.0836 - accuracy: 0.9723 - val_loss: 0.0367 - val_accuracy: 0.9944
Epoch 4/30
400/400 [==============================] - 17s 43ms/step - loss: 0.0835 - accuracy: 0.9732 - val_loss: 0.0441 - val_accuracy: 0.9916
Epoch 5/30
400/400 [==============================] - 17s 42ms/step - loss: 0.0789 - accuracy: 0.9738 - val_loss: 0.0411 - val_accuracy: 0.9925
Epoch 6/30
400/400 [==============================] - 17s 43ms/step - loss: 0.0874 - accuracy: 0.9711 - val_loss: 0.0334 - val_accuracy: 0.9950
Epoch 7/30
400/400 [==============================] - 17s 43ms/step - loss: 0.0793 - accuracy: 0.9721 - val_loss: 0.0311 - val_accuracy:

In [41]:
print(collab_reward)
collab_reward_percent = collab_reward/len(test_img)
print(collab_reward_percent)

11
0.1375


# Game Round 7

In [23]:
# Create an instance of the ModelUpdater class
model_updater = ModelUpdater(weights_path='speaker_agent_updated_5.h5')

# Call the update_model method
model, lr = model_updater.update_model(loaded_model, collab_reward_percent, train_images, train_labels, val_images, val_labels, max_epochs=30)

# Save the updated model
model_path = 'speaker_agent_updated_6.h5'
model.save(model_path)

# Load the updated model
loaded_model = tf.keras.models.load_model(model_path)

# Create an instance of ImagePredictor with appropriate arguments
image_predictor = ImagePredictor(test_img_dir, loaded_model, label_encoder)

# Call the predict_labels method to get the predicted labels
predicted_labels = image_predictor.predict_labels(test_img, target_label)
listener_agent = ListenerAgent(data_path, filenames, shapes)
game_env_lister_model = GameEnvironmentListerModel(test_img_dir, test_img, predicted_labels, listener_agent, data_path, filenames, shapes)
lis_image, lis_pred = game_env_lister_model.get_listener_model_predictions()

collab_reward_calculator = CollaborativeReward(predicted_labels,target_label, lis_pred)
collab_reward, true_conditions, false_conditions = collab_reward_calculator.calculate_reward()



Epoch 1/30
400/400 [==============================] - 21s 44ms/step - loss: 0.0679 - accuracy: 0.9766 - val_loss: 0.0647 - val_accuracy: 0.9862
Epoch 2/30
400/400 [==============================] - 17s 42ms/step - loss: 0.0732 - accuracy: 0.9764 - val_loss: 0.0352 - val_accuracy: 0.9956
Epoch 3/30
400/400 [==============================] - 18s 45ms/step - loss: 0.0760 - accuracy: 0.9746 - val_loss: 0.1511 - val_accuracy: 0.9616
Epoch 4/30
400/400 [==============================] - 17s 43ms/step - loss: 0.0741 - accuracy: 0.9763 - val_loss: 0.0395 - val_accuracy: 0.9925
Epoch 5/30
400/400 [==============================] - 17s 44ms/step - loss: 0.0738 - accuracy: 0.9761 - val_loss: 0.0338 - val_accuracy: 0.9959
Epoch 6/30
400/400 [==============================] - 17s 43ms/step - loss: 0.0649 - accuracy: 0.9779 - val_loss: 0.0336 - val_accuracy: 0.9950
Epoch 7/30
400/400 [==============================] - 17s 43ms/step - loss: 0.0726 - accuracy: 0.9749 - val_loss: 0.0353 - val_accuracy:

In [43]:
print(collab_reward)
collab_reward_percent = collab_reward/len(test_img)
print(collab_reward_percent)

13
0.1625


# Game Round 8

In [25]:
# Create an instance of the ModelUpdater class
model_updater = ModelUpdater(weights_path='speaker_agent_updated_6.h5')

# Call the update_model method
model, lr = model_updater.update_model(loaded_model, collab_reward_percent, train_images, train_labels, val_images, val_labels, max_epochs=30)

# Save the updated model
model_path = 'speaker_agent_updated_7.h5'
model.save(model_path)

# Load the updated model
loaded_model = tf.keras.models.load_model(model_path)

# Create an instance of ImagePredictor with appropriate arguments
image_predictor = ImagePredictor(test_img_dir, loaded_model, label_encoder)

# Call the predict_labels method to get the predicted labels
predicted_labels = image_predictor.predict_labels(test_img, target_label)
listener_agent = ListenerAgent(data_path, filenames, shapes)
game_env_lister_model = GameEnvironmentListerModel(test_img_dir, test_img, predicted_labels, listener_agent, data_path, filenames, shapes)
lis_image, lis_pred = game_env_lister_model.get_listener_model_predictions()

collab_reward_calculator = CollaborativeReward(predicted_labels,target_label, lis_pred)
collab_reward, true_conditions, false_conditions = collab_reward_calculator.calculate_reward()



Epoch 1/30
400/400 [==============================] - 21s 43ms/step - loss: 0.0621 - accuracy: 0.9798 - val_loss: 0.4193 - val_accuracy: 0.8978
Epoch 2/30
400/400 [==============================] - 17s 42ms/step - loss: 0.0662 - accuracy: 0.9777 - val_loss: 0.0305 - val_accuracy: 0.9953
Epoch 3/30
400/400 [==============================] - 17s 42ms/step - loss: 0.0728 - accuracy: 0.9759 - val_loss: 0.0308 - val_accuracy: 0.9956
Epoch 4/30
400/400 [==============================] - 17s 43ms/step - loss: 0.0688 - accuracy: 0.9775 - val_loss: 0.0374 - val_accuracy: 0.9928
Epoch 5/30
400/400 [==============================] - 17s 43ms/step - loss: 0.0709 - accuracy: 0.9755 - val_loss: 0.0449 - val_accuracy: 0.9912
Epoch 6/30
400/400 [==============================] - 17s 42ms/step - loss: 0.0655 - accuracy: 0.9779 - val_loss: 0.0326 - val_accuracy: 0.9956
Epoch 7/30
400/400 [==============================] - 18s 45ms/step - loss: 0.0651 - accuracy: 0.9781 - val_loss: 0.0299 - val_accuracy:

In [45]:
print(collab_reward)
collab_reward_percent = collab_reward/len(test_img)
print(collab_reward_percent)

12
0.15


# Game Round 9

In [27]:
# Create an instance of the ModelUpdater class
model_updater = ModelUpdater(weights_path='speaker_agent_updated_7.h5')

# Call the update_model method
model, lr = model_updater.update_model(loaded_model, collab_reward_percent, train_images, train_labels, val_images, val_labels, max_epochs=30)

# Save the updated model
model_path = 'speaker_agent_updated_8.h5'
model.save(model_path)

# Load the updated model
loaded_model = tf.keras.models.load_model(model_path)

# Create an instance of ImagePredictor with appropriate arguments
image_predictor = ImagePredictor(test_img_dir, loaded_model, label_encoder)

# Call the predict_labels method to get the predicted labels
predicted_labels = image_predictor.predict_labels(test_img, target_label)
listener_agent = ListenerAgent(data_path, filenames, shapes)
game_env_lister_model = GameEnvironmentListerModel(test_img_dir, test_img, predicted_labels, listener_agent, data_path, filenames, shapes)
lis_image, lis_pred = game_env_lister_model.get_listener_model_predictions()

collab_reward_calculator = CollaborativeReward(predicted_labels,target_label, lis_pred)
collab_reward, true_conditions, false_conditions = collab_reward_calculator.calculate_reward()



Epoch 1/30
400/400 [==============================] - 21s 43ms/step - loss: 0.0636 - accuracy: 0.9794 - val_loss: 0.0384 - val_accuracy: 0.9937
Epoch 2/30
400/400 [==============================] - 17s 43ms/step - loss: 0.0571 - accuracy: 0.9807 - val_loss: 0.0371 - val_accuracy: 0.9950
Epoch 3/30
400/400 [==============================] - 17s 43ms/step - loss: 0.0598 - accuracy: 0.9791 - val_loss: 0.1031 - val_accuracy: 0.9747
Epoch 4/30
400/400 [==============================] - 17s 42ms/step - loss: 0.0581 - accuracy: 0.9805 - val_loss: 0.0311 - val_accuracy: 0.9959
Epoch 5/30
400/400 [==============================] - 17s 43ms/step - loss: 0.0565 - accuracy: 0.9807 - val_loss: 0.0324 - val_accuracy: 0.9956
Epoch 6/30
400/400 [==============================] - 17s 43ms/step - loss: 0.0665 - accuracy: 0.9777 - val_loss: 0.0301 - val_accuracy: 0.9956
Epoch 7/30
400/400 [==============================] - 17s 43ms/step - loss: 0.0616 - accuracy: 0.9800 - val_loss: 0.0339 - val_accuracy:

In [48]:
print(collab_reward)
collab_reward_percent = collab_reward/len(test_img)
print(collab_reward_percent)

9
0.1125


# Game round 10

In [29]:
# Create an instance of the ModelUpdater class
model_updater = ModelUpdater(weights_path='speaker_agent_updated_8.h5')

# Call the update_model method
model, lr = model_updater.update_model(loaded_model, collab_reward_percent, train_images, train_labels, val_images, val_labels, max_epochs=30)

# Save the updated model
model_path = 'speaker_agent_updated_9.h5'
model.save(model_path)

# Load the updated model
loaded_model = tf.keras.models.load_model(model_path)

# Create an instance of ImagePredictor with appropriate arguments
image_predictor = ImagePredictor(test_img_dir, loaded_model, label_encoder)

# Call the predict_labels method to get the predicted labels
predicted_labels = image_predictor.predict_labels(test_img, target_label)
listener_agent = ListenerAgent(data_path, filenames, shapes)
game_env_lister_model = GameEnvironmentListerModel(test_img_dir, test_img, predicted_labels, listener_agent, data_path, filenames, shapes)
lis_image, lis_pred = game_env_lister_model.get_listener_model_predictions()

collab_reward_calculator = CollaborativeReward(predicted_labels,target_label, lis_pred)
collab_reward, true_conditions, false_conditions = collab_reward_calculator.calculate_reward()



Epoch 1/30
400/400 [==============================] - 21s 44ms/step - loss: 0.0533 - accuracy: 0.9823 - val_loss: 0.0403 - val_accuracy: 0.9950
Epoch 2/30
400/400 [==============================] - 18s 44ms/step - loss: 0.0546 - accuracy: 0.9830 - val_loss: 0.0374 - val_accuracy: 0.9953
Epoch 3/30
400/400 [==============================] - 17s 42ms/step - loss: 0.0592 - accuracy: 0.9798 - val_loss: 0.0370 - val_accuracy: 0.9956
Epoch 4/30
400/400 [==============================] - 17s 42ms/step - loss: 0.0696 - accuracy: 0.9775 - val_loss: 0.0406 - val_accuracy: 0.9953
Epoch 5/30
400/400 [==============================] - 17s 43ms/step - loss: 0.0561 - accuracy: 0.9819 - val_loss: 0.0400 - val_accuracy: 0.9947
Epoch 6/30
400/400 [==============================] - 17s 42ms/step - loss: 0.0593 - accuracy: 0.9812 - val_loss: 0.0475 - val_accuracy: 0.9903
Epoch 7/30
400/400 [==============================] - 17s 42ms/step - loss: 0.0633 - accuracy: 0.9804 - val_loss: 0.0401 - val_accuracy:

In [30]:
print(collab_reward)
collab_reward_percent = collab_reward/len(test_img)
print(collab_reward_percent)

10
0.125
